In [2]:
from imports import *

In [3]:
df = df[['Date','Round','Venue','Home.Team','Home.Points','Away.Team','Away.Points','Margin']]

In [4]:
team_home_grounds = {
    "Adelaide": ["Adelaide Oval", "Marrara Oval"],
    "Brisbane Lions": ["Gabba", "Cazaly's Stadium"],
    "Carlton": ["M.C.G.", "Docklands"],
    "Collingwood": ["M.C.G."],
    "Essendon": ["M.C.G.", "Docklands"],
    "Fremantle": ["Subiaco", "Perth Stadium"],
    "Geelong": ["Kardinia Park", "M.C.G."],
    "Gold Coast": ["Carrara", "Marrara Oval"],
    "GWS": ["Sydney Showground", "Manuka Oval"],
    "Hawthorn": ["M.C.G.", "York Park"],
    "Melbourne": ["M.C.G.", "Traeger Park"],
    "North Melbourne": ["Docklands", "Bellerive Oval"],
    "Port Adelaide": ["Adelaide Oval"],
    "Richmond": ["M.C.G."],
    "St Kilda": ["Docklands", "Cazaly's Stadium"],
    "Sydney": ["S.C.G.", "Stadium Australia"],
    "West Coast": ["Subiaco", "Perth Stadium"],
    "Footscray": ["Docklands"]
}

In [5]:
def is_true_home(row):
    team = row["Home.Team"]
    venue = row["Venue"]
    return venue in team_home_grounds.get(team, [])

In [6]:
df["True.Home"] = df.apply(is_true_home, axis=1)

C:\Users\matth\AppData\Local\Temp\ipykernel_48264\116173599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["True.Home"] = df.apply(is_true_home, axis=1)


In [7]:
teams = df['Home.Team'].unique()
elo = {team: 1500 for team in teams}

In [8]:
K = 20
home_advantage = 50
starting_elo = 1500
regress_weight = 0.67
neutral_elo = 1500

# --- Initialize Elo ratings ---
teams = pd.unique(df[['Home.Team', 'Away.Team']].values.ravel())
elo = {team: starting_elo for team in teams}

# --- Sort games chronologically ---
df = df.sort_values(by='Date')

# --- Elo history tracker ---
elo_history = []
current_year = None

# --- Elo update loop ---
for _, row in df.iterrows():
    year = row['Year'] if 'Year' in row else pd.to_datetime(row['Date']).year

    # Apply regression at start of new season
    if current_year is not None and year != current_year:
        for team in elo:
            elo[team] = round(regress_weight * elo[team] + (1 - regress_weight) * neutral_elo, 2)

    current_year = year

    home = row['Home.Team']
    away = row['Away.Team']
    home_score = row['Home.Points']
    away_score = row['Away.Points']

    # Pre-game Elo
    home_elo_pre = elo[home]
    away_elo_pre = elo[away]
    home_elo_boosted = home_elo_pre + (home_advantage if row["True.Home"] else 0)

    # Expected win probabilities
    expected_home = 1 / (1 + 10 ** ((away_elo_pre - home_elo_boosted) / 400))
    expected_away = 1 - expected_home

    # Actual result
    result_home = 1 if home_score > away_score else 0
    result_away = 1 - result_home

    # Margin-of-victory multiplier
    margin = abs(home_score - away_score)
    mov_multiplier = (margin + 1) ** 0.8 / (7.5 + 0.006 * abs(home_elo_pre - away_elo_pre))
    update = K * mov_multiplier

    # Elo deltas
    home_elo_delta = update * (result_home - expected_home)
    away_elo_delta = update * (result_away - expected_away)

    # Elo updates
    elo[home] += home_elo_delta
    elo[away] += away_elo_delta

    # Store full match snapshot with rounding
    elo_history.append({
        'Date': row['Date'],
        'Round': row['Round'],
        'Home': home,
        'Away': away,
        'Home Score': home_score,
        'Away Score': away_score,
        'Home Elo Pre': round(home_elo_pre, 2),
        'Away Elo Pre': round(away_elo_pre, 2),
        'Home Elo Pre + Advantage': round(home_elo_boosted, 2),
        'Home Elo Post': round(elo[home], 2),
        'Away Elo Post': round(elo[away], 2),
        'Home Elo Change': round(home_elo_delta, 2),
        'Away Elo Change': round(away_elo_delta, 2),
        'Result': 'Home Win' if result_home == 1 else 'Away Win',
        'Year': year
    })

# --- Convert to DataFrame ---
elo_df = pd.DataFrame(elo_history)

# --- Optional: filter last season ---
last_season = elo_df[elo_df['Year'] == elo_df['Year'].max()]

In [9]:
fixture_2025 = last_season[['Date','Round','Home','Away']]

In [10]:
import requests
import pandas as pd

url = "https://api.squiggle.com.au/?q=games;year=2026"

headers = {
    "User-Agent": "MattAFLModel/1.0 (+https://github.com/matt)",  # <-- important
    "Accept": "application/json"
}

r = requests.get(url, headers=headers)
data = r.json()

df_26 = pd.DataFrame(data["games"])

In [11]:
df_26 = df_26[['localtime','round','venue','hteam','ateam']]

In [12]:
team_map = {
    "Sydney": "Sydney",
    "Gold Coast": "Gold Coast",
    "Greater Western Sydney": "GWS",
    "Brisbane Lions": "Brisbane Lions",
    "St Kilda": "St Kilda",
    "Carlton": "Carlton",
    "Essendon": "Essendon",
    "Western Bulldogs": "Footscray",
    "Geelong": "Geelong",
    "Collingwood": "Collingwood",
    "North Melbourne": "North Melbourne",
    "Melbourne": "Melbourne",
    "Hawthorn": "Hawthorn",
    "Adelaide": "Adelaide",
    "Richmond": "Richmond",
    "Fremantle": "Fremantle",
    "Port Adelaide": "Port Adelaide",
    "West Coast": "West Coast"
}

In [13]:
df_26['Date'] = df_26['localtime'].str[:10]

df_26['Home'] = df_26['hteam'].map(team_map)
df_26['Away'] = df_26['ateam'].map(team_map)

df_26 = df_26.rename(columns={
    'round':'Round'
})

df_26 = df_26[['Date','Round','Home','Away']]

In [14]:
last_season = last_season.sort_values('Date')

# Build long-form team/Elo snapshot from both home and away
home_elos = last_season[['Date', 'Home', 'Home Elo Pre']].rename(columns={'Home': 'Team', 'Home Elo Pre': 'Elo'})
away_elos = last_season[['Date', 'Away', 'Away Elo Pre']].rename(columns={'Away': 'Team', 'Away Elo Pre': 'Elo'})

# Combine and sort
team_elos = pd.concat([home_elos, away_elos]).sort_values('Date')

# Take the first appearance per team
elo_start_2025 = team_elos.groupby('Team').first()['Elo'].round(2).to_dict()

In [15]:
margin_multiplier = 0.075  # Elo-to-margin scaling
elo = elo_start_2025.copy()  # Elo snapshot before Round 1
df_26 = df_26[df_26['Round']<25]

# --- Sort fixture ---
df_26 = df_26.sort_values(by='Date')

sim_results = []

# --- Simulation loop ---
for _, row in df_26.iterrows():
    home = row['Home']
    away = row['Away']
    date = row['Date']
    rnd = row['Round']

    # Elo snapshot
    home_elo_pre = elo[home]
    away_elo_pre = elo[away]
    home_elo_boosted = home_elo_pre + home_advantage

    # Win probability
    expected_home = 1 / (1 + 10 ** ((away_elo_pre - home_elo_boosted) / 400))

    # Predicted margin
    predicted_margin = (home_elo_boosted - away_elo_pre) * margin_multiplier

    # Simulated result (deterministic)
    result_home = 1 if expected_home > 0.5 else 0
    result_away = 1 - result_home

    # Margin-of-victory multiplier
    mov_multiplier = (abs(predicted_margin) + 1) ** 0.8 / (7.5 + 0.006 * abs(home_elo_pre - away_elo_pre))
    update = K * mov_multiplier

    # Elo deltas
    home_elo_delta = update * (result_home - expected_home)
    away_elo_delta = update * (result_away - (1 - expected_home))

    # Elo updates
    elo[home] += home_elo_delta
    elo[away] += away_elo_delta

    # Store result
    sim_results.append({
        'Date': date,
        'Round': rnd,
        'Home': home,
        'Away': away,
        'Home Elo Pre': round(home_elo_pre, 2),
        'Away Elo Pre': round(away_elo_pre, 2),
        'Home Elo Pre + Advantage': round(home_elo_boosted, 2),
        'Win Prob Home': round(expected_home, 3),
        'Predicted Margin': round(predicted_margin, 2),
        'Winner': home if result_home == 1 else away,
        'Home Elo Post': round(elo[home], 2),
        'Away Elo Post': round(elo[away], 2),
        'Home Elo Change': round(home_elo_delta, 2),
        'Away Elo Change': round(away_elo_delta, 2)
    })

# --- Convert to DataFrame ---
sim_df = pd.DataFrame(sim_results)

ladder = sim_df['Winner'].value_counts().reset_index()
ladder.columns = ['Team', 'Wins']
ladder = ladder.sort_values(by='Wins', ascending=False)